# Hier wird geprüft, wie Antworten mit dem abgerufenen Kontext erzeugt werden können:

Als Prompt nutze ich diesen selbsterstellten String:

<|begin_of_text|>
<|start_header_id|>system<|end_header_id|>
Du bist der Assistent der Fachhochschule Südwestfalen (FH-SWF).
Du beantwortest Fragen von Studieninteressierten und Studierenden.

Nutze den bereitgestellten Kontext, um die Frage zu beantworten.
Wenn du die Frage nicht aus dem Kontext beantworten kannst, antworte mit "Das weiß ich leider nicht.".
Erfinde niemals etwas, was nicht aus dem Kontext hervorgeht.

<|start_header_id|>user<|end_header_id|>
Kontext: {context}
Frage: {query}

<|start_header_id|>assistant<|end_header_id|>


Der Prompt berücksichtigt die Special Tokens, welche von Meta zu den llama3.1 Modellen veröffentlicht wurden. Dies steht hier: https://www.llama.com/docs/model-cards-and-prompt-formats/llama3_1/

Hier wurden noch die umformulierten Fragen genutzt, da es lediglich um die ersten Versuche ging, wie Antworten generiert werden können.

Das vollständige Notebook diente lediglich als Test, wie Antworten generiert werden, damit dies in der Evaluation mit den ursprünglichen Fragen durchgeführt werden konnte.

In [1]:
fh_prompt = """
<|begin_of_text|>
<|start_header_id|>system<|end_header_id|>
Du bist der Assistent der Fachhochschule Südwestfalen (FH-SWF).
Du beantwortest Fragen von Studieninteressierten und Studierenden.

Nutze den bereitgestellten Kontext, um die Frage konkret zu beantworten.
Wenn du die Frage nicht aus dem Kontext beantworten kannst, antworte mit "Das weiß ich leider nicht.", sonst nichts.
Erfinde niemals etwas, was nicht aus dem Kontext hervorgeht.

<|start_header_id|>user<|end_header_id|>
Kontext: {context}
Frage: {query}

<|start_header_id|>assistant<|end_header_id|>
"""

In [2]:
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.messages import HumanMessage, AIMessage
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.runnables import Runnable, RunnablePassthrough
from langchain_core.vectorstores import VectorStoreRetriever
from langchain_qdrant import QdrantVectorStore, RetrievalMode
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_ollama import ChatOllama
from qdrant_client.models import Filter, FieldCondition, MatchValue
from langchain.chains.query_constructor.schema import AttributeInfo
from langchain.retrievers.self_query.base import SelfQueryRetriever
from langchain_core.documents import Document

In [3]:
encode_kwargs = {"prompt": "query: "}

In [4]:
embeddings = HuggingFaceEmbeddings(model_name="intfloat/multilingual-e5-large", encode_kwargs=encode_kwargs)

/Users/bastian/Desktop/BachelorProjektAbgabe/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
# Client für Qdrant erstellen
token_collection = "intfloat_multilingual-e5-large_token_based_chunks"
qdrant_url = "http://localhost:6333"
token_client = QdrantVectorStore.from_existing_collection(
    embedding=embeddings,
    retrieval_mode=RetrievalMode.DENSE,
    prefer_grpc=False,
    collection_name=token_collection,
    url=qdrant_url,
)

In [6]:
# 10 Fragen:
fragen = {
    1: "Wann kann ich mich für den Master in Elektrotechnik in Meschede einschreiben?",
    2: "Welche Literatur ist relevant für das Modul Mathematik im Bachelorstudiengang Elektrotechnik in Soest?",
    3: "Welche Fachgebiete sind an der FH vertreten?",
    4: "Ist eine Beurlaubung während des Studiums möglich?",
    5: "Welche Studienmodelle werden angeboten?",
    6: "Wie viele Seiten muss ich in meiner Bachelorarbeit in Wirtschaftsinformatik in Hagen schreiben?",
    7: "Wie sind Portfolioprüfungen im Bachelorstudiengang Elektrotechnik in Soest aufgebaut?",
    8: "Was muss ich beachten, wenn ich bei einer Prüfung krank bin?",
    9: "Was sind die Inhalte im Modul IT-Sicherheit im Bachelorstudiengang Elektrotechnik in Hagen?",
    10: "Wie lange dauern die Klausuren im Bachelorstudiengang Wirtschaftsinformatik in Hagen?"
}

In [7]:
# Filter für die Suche
def create_field_condition(field, value):
    return FieldCondition(
        key=f"metadata.{field}",
        match=MatchValue(value=value)
    )

filters = {
    1: Filter(must=[
        create_field_condition("studiengang", "Elektrotechnik"),
        create_field_condition("standort", "Meschede"),
        create_field_condition("abschluss", "Master")
    ]),
    2: Filter(must=[
        create_field_condition("standort", "Soest"),
        create_field_condition("studiengang", "Elektrotechnik"),
        create_field_condition("abschluss", "Bachelor")
    ]),
    3: Filter(must=[
        create_field_condition("studiengang", "Alle"),
        create_field_condition("standort", "Alle"),
        create_field_condition("abschluss", "Alle")
    ]),
    4: Filter(must=[
        create_field_condition("studiengang", "Alle"),
        create_field_condition("standort", "Alle"),
        create_field_condition("abschluss", "Alle")
    ]),
    5: Filter(must=[
        create_field_condition("studiengang", "Alle"),
        create_field_condition("standort", "Alle"),
        create_field_condition("abschluss", "Alle")
    ]),
    6: Filter(must=[
        create_field_condition("studiengang", "Wirtschaftsinformatik"),
        create_field_condition("standort", "Hagen"),
        create_field_condition("abschluss", "Bachelor")
    ]),
    7: Filter(must=[
        create_field_condition("studiengang", "Elektrotechnik"),
        create_field_condition("standort", "Soest"),
        create_field_condition("abschluss", "Bachelor")
    ]),
    8: Filter(must=[
        create_field_condition("studiengang", "Alle"),
        create_field_condition("standort", "Alle"),
        create_field_condition("abschluss", "Alle")
    ]),
    9: Filter(must=[
        create_field_condition("studiengang", "Elektrotechnik"),
        create_field_condition("standort", "Hagen"),
        create_field_condition("abschluss", "Bachelor")
    ]),
    10: Filter(must=[
        create_field_condition("studiengang", "Wirtschaftsinformatik"),
        create_field_condition("standort", "Hagen"),
        create_field_condition("abschluss", "Bachelor")
    ]),
}

In [8]:
# infos hier: https://python.langchain.com/api_reference/qdrant/qdrant/langchain_qdrant.qdrant.QdrantVectorStore.html#langchain_qdrant.qdrant.QdrantVectorStore.as_retriever
search_kwargs = {"k": 10, "filter": filters[1]} # so kann man dem retriever dann auch noch einen Filter mitgeben
basic_token_based_retriever = token_client.as_retriever(search_kwargs=search_kwargs)

In [9]:
basic_token_based_retriever.invoke(fragen[1])

[Document(metadata={'id': 1021, 'studiengang': 'Elektrotechnik', 'abschluss': 'Master', 'standort': 'Meschede', 'link': 'https://www.fh-swf.de/de/studienangebot/studiengaenge/elektrotechnik_m_eng_/index.php', 'hash': '23d5055fb6686b7f964ad1114b3e41d5d939a5dfe2428fa974f4bd6af87e72a5', '_id': '579aeac7-499f-4213-b1ea-0e9425bb6c64', '_collection_name': 'intfloat_multilingual-e5-large_token_based_chunks'}, page_content='Jahre abhängig von den Studienvoraussetzungen) ### Studienort Meschede ### Beiträge und Gebühren ### Hauptunterrichtssprache Deutsch (für internationale Bewerbende: Level DSH3 / TestDaf5 / C2) ## Vor Beginn des Elektrotechnik Master-Studiums – Voraussetzungen &amp; Bewerbungsverfahren ### Voraussetzungen ##### Für den viersemestrigen Masterstudiengang - Abschluss eines sechssemestrigen Studiums mit mind. 180 Credits in einem der unten aufgeführten Studiengänge mit einer Gesamtnote von mindestens 2,7 (oder mit einer Gesamtnote von mindestens 3,0 und einer Bachelorarbeit mit 

In [10]:
basic_token_based_retriever.invoke("Wo kann ich Wirtschaftsinformatik studieren?")

[Document(metadata={'id': 1043, 'studiengang': 'Elektrotechnik', 'abschluss': 'Master', 'standort': 'Meschede', 'link': 'https://www.fh-swf.de/media/neu_np/hv_2/downloads_der_studiengangsseiten/fb_iw_1/elektrotechnik_m_eng_/FPO_MA_ETM_2017.pdf', 'hash': '85c7b811820a4a3797a21175c07ac8753a8f77facee7c5b5b69af00c3349f9f1', '_id': '1f005ede-71ae-455c-ba98-66675baa78ae', '_collection_name': 'intfloat_multilingual-e5-large_token_based_chunks'}, page_content='| WS 17/18 | | Digitale Produktion | 6 | - | 1 | 1 | 1 | WS 17/18 | | Industriekommunikation | 6 | - | | 1 | 1 | WS 17/18 | | Sensorsysteme | 6 | - | 1 | | 1 | WS 17/18 | | Softwareentwicklung für Echtzeitsysteme | 6 | - | 1 | 1 | 1 | SS 18 | | Spezialgebiete der Aktorik und Mechatronik | 6 | - | | | 1 | WS 17/18 | | Spezialgebiete der Kommunikationstechnik | 6 | - | | 1 | | WS 17/18 | | Spezialgebiete der Business Intelligence | 6 | - | 1 | | | WS 17/18 | | Spezialgebiete der IT | 6 | - | 1 | | | WS 17/18 | | Systemtechnik elektronische

In [11]:
rag_prompt = ChatPromptTemplate.from_template(fh_prompt)

In [12]:
llm = ChatOllama(
    model="llama3.1:8b", # llama3-2:3b ist nicht ausreichend. Auch mit dem Modell hier kann es noch zu Problemen kommen, die durch leistungsstärkere LLMs gelöst werden könnten.
    temperature=0, # Keine Zufälligkeit
    num_ctx=32000 # Kontextfenster
)

In [13]:
# Kontext zu Frage abrufen
first_question_context = basic_token_based_retriever.invoke(fragen[1])
print(first_question_context)

[Document(metadata={'id': 1021, 'studiengang': 'Elektrotechnik', 'abschluss': 'Master', 'standort': 'Meschede', 'link': 'https://www.fh-swf.de/de/studienangebot/studiengaenge/elektrotechnik_m_eng_/index.php', 'hash': '23d5055fb6686b7f964ad1114b3e41d5d939a5dfe2428fa974f4bd6af87e72a5', '_id': '579aeac7-499f-4213-b1ea-0e9425bb6c64', '_collection_name': 'intfloat_multilingual-e5-large_token_based_chunks'}, page_content='Jahre abhängig von den Studienvoraussetzungen) ### Studienort Meschede ### Beiträge und Gebühren ### Hauptunterrichtssprache Deutsch (für internationale Bewerbende: Level DSH3 / TestDaf5 / C2) ## Vor Beginn des Elektrotechnik Master-Studiums – Voraussetzungen &amp; Bewerbungsverfahren ### Voraussetzungen ##### Für den viersemestrigen Masterstudiengang - Abschluss eines sechssemestrigen Studiums mit mind. 180 Credits in einem der unten aufgeführten Studiengänge mit einer Gesamtnote von mindestens 2,7 (oder mit einer Gesamtnote von mindestens 3,0 und einer Bachelorarbeit mit 

In [14]:
from typing import List, Tuple
# Kontexte formatieren vor der Übergabe an das LLM
def keep_relevant_metadata(documents: list[Document]) -> Tuple[List[Document], List[str]]:
    relevant_metadata = [
        "standort",
        "studiengang",
        "abschluss"
    ]
    links = []
    for doc in documents:
        if doc.metadata['link'] not in links:
            links.append(doc.metadata['link'])
        doc.metadata = {key: doc.metadata[key] for key in relevant_metadata}
    return documents, links

def format_page_content(documents: list[Document]) -> str:
    formatted_context = ""
    for doc in documents:
        formatted_context += (f"Standort: {doc.metadata['standort']}, "
                              f"Studiengang: {doc.metadata['studiengang']}, "
                              f"Abschluss: {doc.metadata['abschluss']}\n{doc.page_content}\n\n")
    return formatted_context

In [15]:
first_question_context, links = keep_relevant_metadata(first_question_context)
first_question_context = format_page_content(first_question_context)
print(first_question_context)
for link in links:
    print(link)

Standort: Meschede, Studiengang: Elektrotechnik, Abschluss: Master
Jahre abhängig von den Studienvoraussetzungen) ### Studienort Meschede ### Beiträge und Gebühren ### Hauptunterrichtssprache Deutsch (für internationale Bewerbende: Level DSH3 / TestDaf5 / C2) ## Vor Beginn des Elektrotechnik Master-Studiums – Voraussetzungen &amp; Bewerbungsverfahren ### Voraussetzungen ##### Für den viersemestrigen Masterstudiengang - Abschluss eines sechssemestrigen Studiums mit mind. 180 Credits in einem der unten aufgeführten Studiengänge mit einer Gesamtnote von mindestens 2,7 (oder mit einer Gesamtnote von mindestens 3,0 und einer Bachelorarbeit mit einer besseren Note als 2,3) ##### Für den dreisemestrigen Masterstudiengang - Abschluss eines siebensemestrigen Studiums mit einem Umfang von mind. 210 Credits in einem der unten auf geführten Studiengänge mit einer Gesamtnote von mindestens 2,7 (oder mit einer Gesamtnote von mindestens 3,0 und einer Bachelorarbeit mit einer besseren Note als 2,3) ##

In [16]:
# Kontext und Frage in Prompt einsetzen:
prompt = rag_prompt.invoke({"context": first_question_context, "query": fragen[1]})

In [17]:
# Antwort generieren
answer = llm.invoke(prompt) # Answer ist eine AIMessage aus dem LangChain Framework

In [18]:
print(answer)
print(answer.content)
# Damit sieht man auch, wie viele Token genutzt werden. Das Kontextfenster des LLM für die Antworten kann auf ca. 10000 festgelegt werden. Maximal ca. 5000 Token + ein Paar für Metadaten + Anweisungen aus dem Prompt. Wenn alles auf dem Rechner läuft mit dem Frontend und der Datenbank muss das Kontextfenster eh kleiner dimensioniert sein, da die Speicherkapazität sonst nicht mehr reicht.

content='Für Deutsche und Deutsche gleichgestellte Bewerber*innen können sich im Wintersemester von Mitte Mai bis Ende August und im Sommersemester von Anfang Dezember bis Ende Februar jedes Jahres für den Master-Studiengang in Meschede einschreiben. Für Bewerber*innen aus Nicht-EU-Ländern gelten andere Fristen: Im Wintersemester ist die Einschreibung von Anfang März bis 15. Juli und im Sommersemester von Anfang November bis zum 15. Januar jedes Jahres möglich.' additional_kwargs={} response_metadata={'model': 'llama3.1:8b', 'created_at': '2025-03-10T10:21:04.994246Z', 'done': True, 'done_reason': 'stop', 'total_duration': 45074852250, 'load_duration': 3112675042, 'prompt_eval_count': 6309, 'prompt_eval_duration': 33810000000, 'eval_count': 115, 'eval_duration': 8141000000, 'message': Message(role='assistant', content='', images=None, tool_calls=None)} id='run-8be8f717-bc7b-49c7-b724-5b32acef9489-0' usage_metadata={'input_tokens': 6309, 'output_tokens': 115, 'total_tokens': 6424}
Für D

In [30]:
search_kwargs = {"k": 10}
basic_token_based_retriever = token_client.as_retriever(search_kwargs=search_kwargs)

Durch den Ablauf hat man auch Zugriff auf die abgerufenen Dokumente.
D. h. es können z. B. noch die Metadaten entfernt werden, statt sie mit an das LLM zu geben.
Ggf. können nur die relevanten Metadaten (Standort, Studiengang, Abschluss) an das LLM gegeben werden.
Das könnte hilfreich sein, wenn Fragen ohne Filter gestellt werden, da der Kontext eines jeweiligen Dokumentes dann zugeordnet werden kann.
Werden z. B. Dokumente für zwei gleichnamige Bachelorstudiengänge abgerufen, kann anhand des Standortes in der Frage und in den Metadaten eine Zuordnung erfolgen.
Bei dem Beispiel hier ist das noch möglich, bei 84 Studiengängen aber nicht mehr. Die Anzahl abgerufener Dokumente ist limitiert und das Kontextfenster des LLM auch.

Für mich ist das der bessere Weg. Damit könnten auch die Links ausgegeben werden. Damit kann man auf potenziell relevante Dokumente verweisen.

In [31]:
qa_chain = (
    {
        "context": basic_token_based_retriever,
        "query": RunnablePassthrough(),
    }
    | rag_prompt
    | llm
    | StrOutputParser()
)

Hier werden jetzt leider ganze Dokumente mit den Metadaten an das LLM gegeben. Das ist jedoch nicht mit allen Metadaten sinnvoll.

D. h. es sollte nur der page_content übergeben werden bzw. unnötige Metadaten dürfen nicht mit in den Prompt.

In [32]:
answer = qa_chain.invoke("Wann kann ich mich für den Master in Elektrotechnik in Meschede einschreiben?")

In [35]:
answer

'Für den Masterstudiengang Elektrotechnik an der Fachhochschule Südwestfalen in Meschede gibt es zwei Bewerbungszeiträume pro Jahr. Für Deutsche und Deutsche gleichgestellte Bewerber*innen ist die Einschreibung für das Wintersemester i.d.R. Mitte Mai bis Ende August jeden Jahres möglich, während für das Sommersemester i.d.R. Anfang Dezember bis Ende Februar jeden Jahres der Bewerbungszeitraum geöffnet ist.'

In [36]:
answer = qa_chain.invoke("Welche Literatur ist relevant für das Modul Mathematik im Bachelorstudiengang Elektrotechnik in Soest?")

In [37]:
answer

'Die relevanten Lehrbücher für das Modul "Mathematik" im Bachelor-Studiengang Elektrotechnik in Soest sind:\n\n1. Bolte, Ekkehard: "Elektrische Maschinen", Springer, aktuelle Auflage\n2. Hofmann, Wilfried: "Elektrische Maschinen", Pearson Studium, aktuelle Auflage\n3. Müller, Germar / Ponick, Bernd: "Grundlagen elektrischer Maschinen", Wiley-VCH, aktuelle Auflage\n4. Schröder, Dierk: "Elektrische Antriebe - Regelung von Antriebssystemen", Springer, aktuelle Auflage\n\nDiese Lehrbücher sind im Modulhandbuch für das Modul "Mathematik" im Bachelor-Studiengang Elektrotechnik in Soest aufgeführt.'

In [38]:
answer = qa_chain.invoke("Welche Fachgebiete sind an der FH vertreten?")

In [39]:
answer

'Die Fachhochschule Südwestfalen (FH-SWF) hat mehrere Fachgebiete. Dazu gehören:\n\n* Informatik und Naturwissenschaften\n* Maschinenbau\n* Technische Betriebswirtschaft\n\nDiese Fachgebiete sind an verschiedenen Standorten der FH-SWF vertreten, darunter Meschede, Hagen und Iserlohn.\n\nIn den genannten Dokumenten finden sich auch Belege für die folgenden Studiengänge:\n\n* Wirtschaftsinformatik\n* Elektrotechnik\n\nEs ist jedoch nicht möglich, eine vollständige Liste aller an der FH-SWF vertretenen Fachgebiete oder Studiengänge aus dem bereitgestellten Kontext abzuleiten.'

In [40]:
answer = qa_chain.invoke("Ist eine Beurlaubung während des Studiums möglich?")

In [41]:
answer

'Ja, eine Beurlaubung während des Studiums ist möglich. Nachweislich müssen jedoch wichtige Gründe vorliegen und die Beurlaubungsgründe müssen mindestens die Hälfte des Semesters andauern. Die Beurlaubung kann für ein Semester gewährt werden, beim Freiwilligendienst für zwei Semester.'

In [42]:
answer = qa_chain.invoke("Welche Studienmodelle werden angeboten?")

In [43]:
answer

'Die Fachhochschule Südwestfalen (FH-SWF) bietet verschiedene Studienmodelle an, darunter:\n\n1. Vollzeitstudium: Das Vollzeitstudium ist das meist verbreitete Studienmodell. Die Studenten studieren in der Regel in Präsenz am Standort ihres Studiengangs und legen einen durchschnittlichen Arbeitsaufwand von 40 Stunden pro Woche fest.\n2. Teilzeitstudium: Das Teilzeitstudium bietet die Möglichkeit, neben dem Studium noch andere Verpflichtungen im Leben nachzukommen. Die Studenten studieren in der Regel in Präsenz am Standort ihres Studiengangs, jedoch mit einer reduzierten Modulanzahl pro Semester.\n3. Kooperatives Studienmodell: Das kooperative Studienmodell ermöglicht es den Studenten, während des Studiums praktische Erfahrungen in Unternehmen zu sammeln. Dabei wird das Studium durch Praxisblöcke in der vorlesungsfreien Zeit im Unternehmen ergänzt.\n4. Duales Studium: Das duale Studium ist ein Studienmodell, bei dem Theorie und Praxis eng miteinander verknüpft werden. Die Studenten ler

In [44]:
answer = qa_chain.invoke("Wie viele Seiten muss ich in meiner Bachelorarbeit in Wirtschaftsinformatik in Hagen schreiben?")

In [45]:
answer

'In den Dokumenten, die mir vorliegen, kann ich keine spezifische Anforderung an die Länge einer Bachelorarbeit in Wirtschaftsinformatik finden. Es wird jedoch erwähnt, dass es für das Studium der Wirtschaftsinformatik 180-210 ECTS Punkte gibt und dass ein Bachelor-Studiengang normalerweise 3 Jahre dauert.'

In [46]:
answer = qa_chain.invoke("Wie sind Portfolioprüfungen im Bachelorstudiengang Elektrotechnik in Soest aufgebaut?")

In [47]:
answer

'Portfolioprüfungen im Bachelorstudiengang Elektrotechnik in Soest sind laut den bereitgestellten Dokumenten keine explizite Prüfungsform. Es wird jedoch erwähnt, dass das Modul "Portfolio" als Wahlpflichtmodul angeboten wird und in der Fachprüfungsordnung (FPO) 2020 im Teil 2 als § 14 definiert ist.\n\nIn diesem Abschnitt heißt es: "§ 14 Portfolio\n\n(1) Das Portfolio besteht aus einer Sammlung von Dokumenten, die die Leistungen des Studierenden während der gesamten Studiendauer dokumentieren. (2) Die Dokumente können in Form von Schriftdokumenten, Bildern oder Videos erstellt werden und müssen eine klare Struktur aufweisen."\n\nEs wird jedoch nicht spezifiziert, wie die Prüfung des Portfolios durchgeführt wird.'

In [48]:
answer = qa_chain.invoke("Was muss ich beachten, wenn ich bei einer Prüfung krank bin?")

In [49]:
answer

'Wenn Sie bei einer Prüfung krank sind, sollten Sie sich unverzüglich an einen Arzt wenden und ein ärztliches Attest ausstellen lassen. Dieses Attest muss spätestens innerhalb von drei Werktagen nach der Prüfung beim Studierenden-Servicebüro eingereicht werden.\n\nEs ist wichtig zu beachten, dass Sie sich nicht ohne ärztliche Bescheinigung von einer Prüfung zurückziehen sollten. Wenn Sie bereits an einer Prüfung teilgenommen haben und sich dann als krank erweisen, müssen Sie ein qualifiziertes Attest vorlegen, das besagt, dass Sie zum Zeitpunkt der Prüfung prüfungsunfähig waren.\n\nEs ist auch wichtig zu beachten, dass die Vorlage eines ärztlichen Attests nicht automatisch zur Genehmigung des Rücktritts führt. Der Prüfungsausschuss muss den Antrag auf Rücktritt noch genehmigen.\n\nWenn Sie sich entscheiden, von einer Prüfung zurückzutreten, sollten Sie dies so schnell wie möglich tun, da der Rücktritt bis 7 Tage vor der Prüfung ohne weitere Nachweise völlig unproblematisch ist.'

In [50]:
answer = qa_chain.invoke("Was sind die Inhalte im Modul IT-Sicherheit im Bachelorstudiengang Elektrotechnik in Hagen?")

In [51]:
answer

'Das Modul IT-Sicherheit soll die Studierenden für die Gefahren sensibilisieren, die aus einer absichtlich oder unabsichtlich herbeigeführten Einwirkung resultieren. Ausgehend von der Definition der Grundwerte der IT-Sicherheit erfolgt zunächst eine Betrachtung der aus Malware resultierenden Gefährdungen. Anschließend geht das Modul auf die Gefahren bei der Internetkommunikation und die Gefährdungen von Webanwendungen ein. Zu den identifizierten Gefährdungen werden, dem aktuellem Stand der Technik entsprechend, mögliche Gegenmaßnahmen identifiziert, die zur Abwehr oder zumindest zur Milderung der Gefahren eingesetzt werden können. Hierbei wird besonders auf den Einsatz von kryptographischen Verfahren zur Absicherung der Authentizität, Vertraulichkeit und Integrität von Informationen eingegangen. Ergänzend hierzu wird auf weitere ausgewählte infrastrukturelle Maßnahmen eingegangen, die einen Einfluss auf die Sicherheit der Informationstechnologie haben.'

In [52]:
answer = qa_chain.invoke("Wie lange dauern die Klausuren im Bachelorstudiengang Wirtschaftsinformatik in Hagen?")

In [53]:
answer

'Die Klausuren im Bachelorstudiengang Wirtschaftsinformatik in Hagen dauern jeweils 45-60 Minuten.'

Grundsätzlich funktioniert damit der Abruf der Dokumente und das Generieren von Antworten.